# Imports

In [1]:
import numpy as np
import pandas as pd

# Load in Data

In [2]:
pickle_to_load = "../../FantasyData/data-frames/df_wr_college_2000_2023.pkl"

In [3]:
df_pos = pd.read_pickle(pickle_to_load)

# Clean Up Data

In [4]:
df_clean = df_pos.copy().dropna(subset=["draft_ovr"])

In [5]:
df = df_clean[["draft_ovr", "player_name", "tier"]]

# Get Draft Capital Success Rate By Round

Defining pick ranges:

In [18]:
bins = [0, 3, 5, 10, 15, 20, 32, float('inf')]  # This defines 'Top 3', 'Top 5', 'Top 10', and 'Others'
labels = ['1-3', '4-5', '6-10', '11-15', '16-20', '21-32', 'Others']

Categorize each player into a pick range:

In [19]:
# Step 2: Categorize each player into a pick range
df['pick_range'] = pd.cut(df['draft_ovr'], bins=bins, labels=labels, right=True)

/tmp/ipykernel_25427/185501029.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pick_range'] = pd.cut(df['draft_ovr'], bins=bins, labels=labels, right=True)


In [20]:
pick_ranges = df[['player_name', 'pick_range']].drop_duplicates()

In [21]:
career_best_tier = df.groupby('player_name')['tier'].min().reset_index()

In [22]:
tier_counts = pd.merge(career_best_tier, pick_ranges, on='player_name')

In [23]:
total_players_per_range = tier_counts.groupby('pick_range')['player_name'].count().reset_index(name='total_players')

/tmp/ipykernel_25427/3307901133.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  total_players_per_range = tier_counts.groupby('pick_range')['player_name'].count().reset_index(name='total_players')


In [24]:
players_in_tier = tier_counts.groupby(['pick_range', 'tier']).size().reset_index(name='players_in_tier')

/tmp/ipykernel_25427/1144034013.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  players_in_tier = tier_counts.groupby(['pick_range', 'tier']).size().reset_index(name='players_in_tier')


In [25]:
# Merge to get the total players per pick range
merged_data = pd.merge(players_in_tier, total_players_per_range, on='pick_range')

# Calculate the odds as a percentage
merged_data['percentage'] = (merged_data['players_in_tier'] / merged_data['total_players']) * 100

In [26]:
# Filter out rows where 'Percentage' is 0
final_df = merged_data.copy().loc[merged_data['percentage'] != 0]

In [27]:
final_df

,pick_range,tier,players_in_tier,total_players,percentage
0,1-3,1,4,4,100.000000
19,4-5,1,3,6,50.000000
20,4-5,2,1,6,16.666667
21,4-5,3,2,6,33.333333
38,6-10,1,6,14,42.857143
39,6-10,2,1,14,7.142857
40,6-10,3,4,14,28.571429
41,6-10,4,1,14,7.142857
44,6-10,7,1,14,7.142857
48,6-10,11,1,14,7.142857


# Analysis